<a href="https://colab.research.google.com/github/robbybrodie/time_as_computation_cost/blob/main/notebooks/01_Causal_Diamond_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Causal Diamond Experiment

This notebook explores lightcone/diamond lattice construction and propagation dynamics.

## Theory

- **Lattice Structure**: Nodes at (t,x) where |x| ≤ t ≤ D
- **Edges**: (t,x) → (t+1, x±1) (light cone propagation)
- **Capacity Gradient**: N(t,x) = clip(1 - α*(|x|+t)/D, 0, 1)
- **Propagation**: Random walks weighted by node capacity

## Setup

In [ ]:
# Colab bootstrap
REPO_URL = "https://github.com/robbybrodie/time_as_computation_cost.git"
REPO_NAME = "time_as_computation_cost"

import pathlib
if not pathlib.Path(REPO_NAME).exists():
    !git clone $REPO_URL
%cd $REPO_NAME

if pathlib.Path("pyproject.toml").exists():
    !pip install -e .

In [ ]:
from experiments.run_causal_diamond import main
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import sys

# Ensure we can import the modules
repo_root = Path().resolve()
sys.path.insert(0, str(repo_root / "src"))

from tacc.lattices.diamond import run_demo, run_experiment

## Basic Experiment

In [ ]:
# Run the basic experiment
results = main()

print("Experiment completed successfully!")
print(f"Node count: {results['metrics']['node_count']}")
print(f"Edge count: {results['metrics']['edge_count']}")
print(f"Paths to top: {results['metrics']['actual_paths_top']} (theoretical: {results['metrics']['theoretical_paths_top']})")

## Interactive Demo

In [ ]:
# Create and display demo visualization
fig = run_demo(depth=8, alpha=0.15, show_front=True)
plt.show()

## Parameter Exploration

In [ ]:
# Explore different alpha values
alphas = [0.0, 0.1, 0.2, 0.5]
depth = 8

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.flatten()

for i, alpha in enumerate(alphas):
    # Run experiment for this alpha
    exp_results = run_experiment(depth=depth, alpha=alpha)
    
    # Plot front profile
    times = exp_results['front_data']['times']
    mean_pos = exp_results['front_data']['mean_positions']
    std_pos = exp_results['front_data']['std_positions']
    
    axes[i].plot(times, mean_pos, 'b-', linewidth=2, label='Mean |x|')
    axes[i].fill_between(times, 
                        np.array(mean_pos) - np.array(std_pos),
                        np.array(mean_pos) + np.array(std_pos),
                        alpha=0.3)
    
    # Theoretical expectation
    theoretical = np.sqrt(np.array(times) * 0.5)
    axes[i].plot(times, theoretical, 'r--', alpha=0.7, label='Theoretical sqrt(t/2)')
    
    axes[i].set_title(f'α = {alpha} (deviation: {exp_results["metrics"]["front_symmetry_deviation"]:.3f})')
    axes[i].set_xlabel('Time t')
    axes[i].set_ylabel('Mean |x|')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Depth Scaling Analysis

In [ ]:
# Analyze how metrics scale with depth
depths = [5, 8, 10, 12]
alpha = 0.1

node_counts = []
edge_counts = []
path_counts = []
theoretical_paths = []

for depth in depths:
    exp_results = run_experiment(depth=depth, alpha=alpha)
    node_counts.append(exp_results['metrics']['node_count'])
    edge_counts.append(exp_results['metrics']['edge_count'])
    path_counts.append(exp_results['metrics']['actual_paths_top'])
    theoretical_paths.append(exp_results['metrics']['theoretical_paths_top'])

# Plot scaling relationships
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 10))

# Node count scaling
ax1.plot(depths, node_counts, 'bo-', linewidth=2, markersize=8)
# Theoretical: nodes = (D+1)^2 approximately
theoretical_nodes = [(d+1)**2 for d in depths]
ax1.plot(depths, theoretical_nodes, 'r--', alpha=0.7, label='~D²')
ax1.set_xlabel('Depth D')
ax1.set_ylabel('Node Count')
ax1.set_title('Node Count vs Depth')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Edge count scaling
ax2.plot(depths, edge_counts, 'go-', linewidth=2, markersize=8)
ax2.set_xlabel('Depth D')
ax2.set_ylabel('Edge Count')
ax2.set_title('Edge Count vs Depth')
ax2.grid(True, alpha=0.3)

# Path count comparison
ax3.semilogy(depths, path_counts, 'ro-', linewidth=2, markersize=8, label='Actual')
ax3.semilogy(depths, theoretical_paths, 'b--', linewidth=2, label='Theoretical ~2^D')
ax3.set_xlabel('Depth D')
ax3.set_ylabel('Path Count (log scale)')
ax3.set_title('Path Scaling')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Node/edge ratio
ratios = [e/n for e, n in zip(edge_counts, node_counts)]
ax4.plot(depths, ratios, 'mo-', linewidth=2, markersize=8)
ax4.set_xlabel('Depth D')
ax4.set_ylabel('Edge/Node Ratio')
ax4.set_title('Graph Density')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Scaling Summary:")
for i, depth in enumerate(depths):
    print(f"D={depth}: {node_counts[i]} nodes, {edge_counts[i]} edges, {path_counts[i]} paths")

## Key Insights

1. **Lattice Growth**: Node count scales as ~D², edge count scales similarly
2. **Path Explosion**: Path count grows exponentially with depth
3. **Capacity Effect**: Higher α reduces front spreading (more constraint)
4. **Random Walk**: Front profile follows √t scaling as expected

## Physical Interpretation

- **Computational Capacity N(t,x)**: Controls "computational resources" available at spacetime point
- **Front Propagation**: Information/computation spreading through spacetime
- **Critical Points**: Could represent phase transitions in computational capacity

**Note**: This is a mathematical model without established physics foundation!